In [1]:
import simpy
import random

In [2]:
"""
Modify Attempt 100

"""

RANDOM_SEED = 42
NEW_CUSTOMERS = 5  # Total number of customers
INTERVAL_CUSTOMERS = 10.0  # Generate new customers roughly every x seconds
MIN_PATIENCE = 1  # Min. customer patience
MAX_PATIENCE = 3  # Max. customer patience
min_count_limit = 1
max_count_limit = 5


class Machine:

    def __init__(self, env, resource_dict, machine_id, store):
        self.env = env
        self.resource_dict = resource_dict
        self.machine_id = machine_id
        self.resource = self.resource_dict[self.machine_id]
        self.store = store


    def do_something(self, item):
        arrive = env.now
        with self.resource.request() as req:
            yield req
            yield self.env.timeout(3)
        print('%7.4f Counter%00d: Opened' % (arrive, self.machine_id))
        print(f"{arrive} - counter: {self.machine_id} - deal with item: {item}")

    def run(self):
        while True:
            item = yield self.store.get()
            self.env.process(self.do_something(item))



def source(env, number, interval, resource_dict):
    """Source generates customers randomly"""
    for i in range(number):
        c = customer(env, 'Customer%02d' % i, resource_dict, time_in_bank=12.0)
        env.process(c)
        t = random.expovariate(1.0 / interval)
        yield env.timeout(t)

def machine_count(env, numm, interval, machines):
    machines = [ Machine(env, resource_dict, i, store) for i in range(3)]

    for machine in machines:
        env.process(machine.run())
        print('Counter: Opened %s' % numm)
    t = random.expovariate(1.0 / interval)    
    yield env.timeout(t)



def customer(env, name, resource_dict, time_in_bank):
    """Customer arrives, is served and leaves."""
    arrive = env.now
    # print('%7.4f Counter : Opened' % (arrive))

    print('%7.4f %s: Here I am' % (arrive, name))

    with resource_dict.request() as req:
        patience = random.uniform(MIN_PATIENCE, MAX_PATIENCE)
        # Wait for the counter or abort at the end of our tether
        results = yield req | env.timeout(patience)

        wait = env.now - arrive
        
        if req in results:
            # We got to the counter
            
            print('%7.4f %s: Waited %6.3f' % (env.now, name, wait))

            tib = random.expovariate(1.0 / time_in_bank)
            yield env.timeout(tib)
            print('%7.4f %s: Served by Counter ' % (env.now, name))
            

        else:
            # We reneged
            print('%7.4f %s: About to give up after %6.3f' % (env.now, name, wait))
            # print('%7.4f Counter : Opened' % (arrive))



# Setup and start the simulation
print('Bank renege modified')
random.seed(RANDOM_SEED)


if __name__ == '__main__':

    env = simpy.Environment()
    store = simpy.Store(env)
    # store.items = [f'customer{i}' for i in range(5)]

    resource_dict = {}

    for i in range(6):
        resource_dict[i] = simpy.Resource(env, capacity = 2)

    

# Start processes and run
counter = simpy.Resource(env, capacity = 1)
env.process(source(env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, counter))

env.process(machine_count(env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, machines=6))

# env.process(machine.run())
# env,process()
# machines = [ Machine(env, resource_dict, i, store) for i in range(3)]

# for machine in machines:
#     env.process(machine.run())


env.run()

Bank renege modified
Counter: Opened 5
Counter: Opened 5
Counter: Opened 5
 0.0000 Customer00: Here I am
 0.0000 Customer00: Waited  0.000
 3.0310 Customer00: Served by Counter 
10.2006 Customer01: Here I am
10.2006 Customer01: Waited  0.000
23.5365 Customer02: Here I am
24.4461 Customer03: Here I am
24.7486 Customer04: Here I am
25.3804 Customer02: About to give up after  1.844
25.8016 Customer04: About to give up after  1.053
25.8833 Customer03: About to give up after  1.437
36.9281 Customer01: Served by Counter 


In [5]:
"""
Modify Attempt 127

"""
RANDOM_SEED = 42
PT_MEAN = 10.0         # Avg. processing time in minutes
PT_SIGMA = 2.0         # Sigma of processing time
JOB_DURATION = 30.0    # Duration of other jobs in minutes
NUM_MACHINES = 10      # Number of machines in the machine shop
REPAIR_TIME = 20


""""""""
RANDOM_SEED = 42
NEW_CUSTOMERS = 5  # Total number of customers
INTERVAL_CUSTOMERS = 10.0  # Generate new customers roughly every x seconds
MIN_PATIENCE = 1  # Min. customer patience
MAX_PATIENCE = 3  # Max. customer patience


def time_per_part():
    """Return actual processing time for a concrete part."""
    return random.normalvariate(PT_MEAN, PT_SIGMA)




class Machine(object):
    """A machine produces parts and my get broken every now and then.

    If it breaks, it requests a *repairman* and continues the production
    after the it is repaired.

    A machine has a *name* and a numberof *parts_made* thus far.

    """
    def __init__(self, env, name, repairman):
        self.env = env
        self.name = name
        # self.parts_made = 0
        self.broken = False

        # Start "working" and "break_machine" processes for this machine.
        self.process = env.process(self.working(repairman))
        # env.process(self.break_machine())

    def working(self, repairman):
        """Produce parts as long as the simulation runs.

        While making a part, the machine may break multiple times.
        Request a repairman when this happens.

        """
        while True:
            # Start making a new part
            done_in = time_per_part()
            while done_in:
                try:
                    # Working on the part
                    start = self.env.now
                    yield self.env.timeout(done_in)
                    done_in = 0  # Set to 0 to exit while loop.

                except simpy.Interrupt:
                    self.broken = True
                    done_in -= self.env.now - start  

                    with repairman.request(priority=1) as req:
                        yield req
                        yield self.env.timeout(REPAIR_TIME)

                    self.broken = False




def source(env, number, interval, counter):
    """Source generates customers randomly"""
    for i in range(number):
        c = customer(env, 'Customer%02d' % i, counter, time_in_bank=12.0)
        env.process(c)
        t = random.expovariate(1.0 / interval)
        yield env.timeout(t)

    # for i in range(counter):
    #     cnt = customer(env, 'Counter%02d' % i, counter, time_in_bank=12.0)
    #     env.process(cnt)


def customer(env, name, counter, time_in_bank):
    """Customer arrives, is served and leaves."""
    arrive = env.now
    print('%7.4f %s: Here I am' % (arrive, name))

    with counter.request() as req:
        patience = random.uniform(MIN_PATIENCE, MAX_PATIENCE)
        # Wait for the counter or abort at the end of our tether
        results = yield req | env.timeout(patience)

        wait = env.now - arrive

        if req in results:
            # We got to the counter
            print('%7.4f %s: Waited %6.3f' % (env.now, name, wait))
            # print('%7.4f %s: Opened %6.3f' % (env.now, name, wait))


            tib = random.expovariate(1.0 / time_in_bank)
            yield env.timeout(tib)
            print('%7.4f %s: Served by ' % (env.now, name))
            

        else:
            # We reneged
            print('%7.4f %s: About to give up after %6.3f' % (env.now, name, wait))
            repairman = simpy.PreemptiveResource(env, capacity=1)
            machines = [Machine(env, 'Counter%d' % i, repairman) for i in range(NUM_MACHINES)]
            for machine in machines:
                print('%s made 0 parts.' % (machine.name))


# Setup and start the simulation
print('Bank renege modified')
random.seed(RANDOM_SEED)
env = simpy.Environment()

# Start processes and run
counter = simpy.Resource(env, capacity = 1)
env.process(source(env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, counter))
# Execute!


print(counter)
# Setup and start the simulation
random.seed(RANDOM_SEED)  # This helps reproducing the results

env.run(until=10)


# cnt = [customer(env, 'Customer%02d' % i, counter, time_in_bank=12.0) for i in range(NUM_MACHINES)]
# for i in cnt:  
#     print('%7.4f %s: Opened' % (cnt))



Bank renege modified
 0.0000 Customer00: Here I am
 0.0000 Customer00: Waited  0.000
 3.8595 Customer00: Served by 
